In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy
import pickle
from glob import glob
from typing import Any, Dict, List, Tuple, Union
import pandas as pd

"""Change to the data folder"""
new_path = "./new_train/new_train"
test_path = './new_val_in/new_val_in'
subset_test_path = './new_train/train_subset'
# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)
test_dataset = ArgoverseDataset(data_path=test_path)

### Create a loader to enable batch processing

In [3]:
batch_sz = 4

def train_agents_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = numpy.concatenate([numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch])
    out = numpy.concatenate([numpy.dstack([scene['p_out'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch])
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]

def train_all_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = numpy.concatenate([numpy.dstack([scene['p_in'][['dummy' not in word for word in scene['track_id'][:,0,0]],:,:]]) for scene in batch])
    out = numpy.concatenate([numpy.dstack([scene['p_out'][['dummy' not in word for word in scene['track_id'][:,0,0]],:,:]]) for scene in batch])
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]
    
def test_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = numpy.concatenate([numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch])
    inp = torch.Tensor(inp)
    idx = [numpy.dstack([scene['scene_idx']]) for scene in batch]
    return inp, idx
    
train_agent_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=train_agents_collate, num_workers=0)

train_all_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=train_all_collate, num_workers=0)

test_loader = DataLoader(test_dataset,batch_size=batch_sz, shuffle = True, collate_fn=test_collate, num_workers=0)

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class EncoderRNN(nn.Module):
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    def __init__(self,
                 input_size = 2,
                 embedding_size = 8,
                 hidden_size = 16):
        
        super(EncoderRNN, self).__init__()
        
        dropnum = 0.35
        
        self.hidden_size = hidden_size
        self.linear = nn.Linear(input_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)
        self.drop1 = nn.Dropout(dropnum)
        self.drop2 = nn.BatchNorm1d(embedding_size)
        
    def forward(self, x, hidden):
        embedded = F.relu(self.linear(x))
        embedded = self.drop2(embedded)
        hidden = self.lstm(embedded, hidden)
        return hidden


class DecoderRNN(nn.Module):
    """Decoder Network."""
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    def __init__(self, embedding_size=8, hidden_size=16, output_size=2):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.linear1 = nn.Linear(output_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear1(x))
        hidden = self.lstm(embedded, hidden)
        output = self.linear2(hidden[0])
        return output, hidden


In [5]:
from tqdm import tqdm_notebook as tqdm

def train(encoder, decoder, device, train_loader, encoder_optimizer, decoder_optimizer, epoch, log_interval=10000):    
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    iterator = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    criterion = nn.MSELoss()
    
    for i_batch, sample_batch in enumerate(train_loader):
        
        inp, out = sample_batch
#         print(inp.shape)
        # preprocessing more ????
#         inp = inp[:,0,:,:]
#         out = out[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(inp.shape[0]):
            x_offset.append(inp[i][0][0].detach().clone())
            y_offset.append(inp[i][0][1].detach().clone())
    
        for j in range(inp.shape[0]):
            for i in range(inp.shape[1]):
                inp[j][i][0] = inp[j][i][0] - x_offset[j]
                inp[j][i][1] = inp[j][i][1] - y_offset[j]

        #outoput whatever
        for j in range(out.shape[0]):
            for i in range(out.shape[1]):
                out[j][i][0] = out[j][i][0] - x_offset[j]
                out[j][i][1] = out[j][i][1] - y_offset[j]
        
        _input, target = inp.to(device), out.to(device)
        
        encoder.train()
        decoder.train()
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        
        #encoder 
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        output_length = target.shape[1]
        feature_len = _input.shape[2]
        input_shape = _input.shape[2]
        
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
        
        loss = 0
        
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(target.shape).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Update loss
            loss += torch.sqrt(criterion(decoder_output[:, :2], target[:, di, :2]))

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        # Get average loss for pred_len
        loss = loss / 30

        # Backpropagate
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
        
#         file1 = open("loss_steps.txt", "a")  # append mode
#         file1.write(str(loss.item()) + ",")
#         file1.close()
        
#       output = model(data)
#         loss = MSELoss(output, target)
        counter += 1
        iterator.set_postfix(loss=(loss.item()))
    

In [6]:
device = "cuda"
encoder = EncoderRNN(input_size=2)
decoder = DecoderRNN(output_size=2)

encoder = nn.DataParallel(encoder)
decoder = nn.DataParallel(decoder)

encoder.to(device)
decoder.to(device)

encoder_optimizer = torch.optim.Adam(encoder.parameters())
decoder_optimizer = torch.optim.Adam(decoder.parameters())

num_epoch = 6

for epoch in range(1, num_epoch + 1):
    train(encoder, decoder, device, train_all_loader, encoder_optimizer, decoder_optimizer, epoch)
#         train(encoder, decoder, device, train_agent_loader, encoder_optimizer, decoder_optimizer, epoch)
#         predict(model, device, test_loader)

In [7]:
def infer_absolute(
        test_loader: torch.utils.data.DataLoader,
        encoder: EncoderRNN,
        decoder: DecoderRNN,
#         start_idx: int,
#         forecasted_save_dir: str,
#         model_utils: ModelUtils,
):
    """Infer function for non-map LSTM baselines and save the forecasted trajectories.
    
    referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting
    
    Args:
        test_loader: DataLoader for the test set
        encoder: Encoder network instance
        decoder: Decoder network instance
        start_idx: start index for the current joblib batch
        forecasted_save_dir: Directory where forecasted trajectories are to be saved
        model_utils: ModelUtils instance

    """
    
    forecasted_trajectories = {}

    for i, (_input, idx) in enumerate(test_loader):
        
#         _input = _input[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(_input.shape[0]):
            x_offset.append(_input[i][0][0].detach().clone())
            y_offset.append(_input[i][0][1].detach().clone())
    
        for j in range(_input.shape[0]):
            for i in range(_input.shape[1]):
                _input[j][i][0] = _input[j][i][0] - x_offset[j]
                _input[j][i][1] = _input[j][i][1] - y_offset[j]

        _input = _input.to(device)

        # Set to eval mode
        encoder.eval()
        decoder.eval()

        # Encoder
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        input_shape = _input.shape[2]

        # Initialize encoder hidden state
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
       
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(
            (batch_size, 30, 2)).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        for i in range(30):
            for j in range(batch_size):
                decoder_outputs[j,i,0] = decoder_outputs[j,i,0] + x_offset[j]
                decoder_outputs[j,i,1] = decoder_outputs[j,i,1] + y_offset[j]
            
                if (idx[j][0][0][0] in forecasted_trajectories):
                    forecasted_trajectories[idx[j][0][0][0]].append(decoder_outputs[j,i,:].tolist())
                else:
                    forecasted_trajectories[idx[j][0][0][0]] = [decoder_outputs[j,i,:].tolist()]
                
    return(forecasted_trajectories)

In [8]:
output = infer_absolute(test_loader, encoder, decoder)

In [9]:
import pandas as pd
df = pd.DataFrame.from_dict(output, orient='index')
df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
20855,"[1793.7520751953125, 401.72747802734375]","[1795.0802001953125, 402.8623046875]","[1796.517333984375, 403.8125305175781]","[1797.781982421875, 404.7039489746094]","[1799.014404296875, 405.64422607421875]","[1800.232177734375, 406.6121520996094]","[1801.4281005859375, 407.59197998046875]","[1802.60107421875, 408.5738830566406]","[1803.751708984375, 409.55096435546875]","[1804.8814697265625, 410.51922607421875]",...,"[1816.2139892578125, 420.234375]","[1817.158935546875, 420.9725341796875]","[1818.096923828125, 421.682861328125]","[1819.03173828125, 422.3697509765625]","[1819.9674072265625, 423.0394287109375]","[1820.9083251953125, 423.6994323730469]","[1821.858154296875, 424.3583679199219]","[1822.8203125, 425.02557373046875]","[1823.79638671875, 425.71075439453125]","[1824.7867431640625, 426.42388916015625]"
36575,"[728.4514770507812, 1011.6459350585938]","[729.3147583007812, 1019.6922607421875]","[727.3152465820312, 1023.5125732421875]","[724.5400390625, 1021.8151245117188]","[723.6094360351562, 1021.7288818359375]","[723.0615234375, 1022.9796752929688]","[722.6148071289062, 1024.515625]","[722.1616821289062, 1026.1446533203125]","[721.6896362304688, 1027.86767578125]","[721.2077026367188, 1029.7239990234375]",...,"[719.0806274414062, 1057.4608154296875]","[718.5123291015625, 1057.9271240234375]","[718.0506591796875, 1058.2591552734375]","[717.6188354492188, 1058.3677978515625]","[717.0728149414062, 1058.025634765625]","[716.1522216796875, 1056.7628173828125]","[714.5380859375, 1053.9747314453125]","[712.31982421875, 1049.83447265625]","[710.3369140625, 1045.9752197265625]","[709.1470947265625, 1043.5684814453125]"
5898,"[580.7723388671875, 1508.721435546875]","[581.4405517578125, 1507.7623291015625]","[581.3950805664062, 1506.460693359375]","[581.781494140625, 1505.2735595703125]","[582.0285034179688, 1504.03759765625]","[582.3067016601562, 1502.8211669921875]","[582.5493774414062, 1501.5836181640625]","[582.7716064453125, 1500.322265625]","[582.9674682617188, 1499.03955078125]","[583.1390991210938, 1497.7471923828125]",...,"[583.7652587890625, 1484.3873291015625]","[583.7144165039062, 1483.160400390625]","[583.646484375, 1481.90966796875]","[583.5626831054688, 1480.63330078125]","[583.4647827148438, 1479.33056640625]","[583.3553466796875, 1478.0025634765625]","[583.2377319335938, 1476.652587890625]","[583.1160278320312, 1475.285888671875]","[582.9951171875, 1473.909912109375]","[582.8803100585938, 1472.5343017578125]"
22731,"[844.4381103515625, 1483.102294921875]","[845.0108032226562, 1482.0360107421875]","[844.9710083007812, 1480.514404296875]","[845.1663208007812, 1479.039794921875]","[845.310791015625, 1477.525146484375]","[845.4533081054688, 1475.99609375]","[845.5704956054688, 1474.441162109375]","[845.6622314453125, 1472.881103515625]","[845.7312622070312, 1471.34619140625]","[845.7833251953125, 1469.8616943359375]",...,"[845.629150390625, 1455.358642578125]","[845.543701171875, 1453.9456787109375]","[845.4498901367188, 1452.50146484375]","[845.3516845703125, 1451.03271484375]","[845.254638671875, 1449.549560546875]","[845.165771484375, 1448.0657958984375]","[845.0930786132812, 1446.598388671875]","[845.0450439453125, 1445.166015625]","[845.0292358398438, 1443.7874755859375]","[845.0512084960938, 1442.479248046875]"
30778,"[697.5593872070312, 1155.1175537109375]","[697.0890502929688, 1155.2747802734375]","[696.5968627929688, 1154.7718505859375]","[696.2247924804688, 1154.50537109375]","[695.8160400390625, 1154.1275634765625]","[695.3809814453125, 1153.7880859375]","[694.8981323242188, 1153.4185791015625]","[694.369140625, 1153.0423583984375]","[693.7898559570312, 1152.64794921875]","[693.162353515625, 1152.2393798828125]",...,"[685.6744384765625, 1147.8341064453125]","[685.0811767578125, 1147.5020751953125]","[684.496337890625, 1147.176513671875]","[683.9175415039062, 1146.8558349609375]","[683.3430786132812, 1146.5391845703125]","[682.7717895507812, 1146.2

In [10]:
for i in range(30):
    df[['v{}'.format((i*2)+1), 'v{}'.format((i*2)+2)]] = pd.DataFrame(df.get(i).tolist(), index=df.index)

In [11]:
import numpy as np
dropped_cols = list(np.arange(30))
df2 = df.drop(dropped_cols, axis=1)
df2.index.name = 'ID'
df2

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60
ID,,,,,,,,,,,,,,,,,,,,,
20855,1793.752075,401.727478,1795.080200,402.862305,1796.517334,403.812531,1797.781982,404.703949,1799.014404,405.644226,...,1820.908325,423.699432,1821.858154,424.358368,1822.820312,425.025574,1823.796387,425.710754,1824.786743,426.423889
36575,728.451477,1011.645935,729.314758,1019.692261,727.315247,1023.512573,724.540039,1021.815125,723.609436,1021.728882,...,716.152222,1056.762817,714.538086,1053.974731,712.319824,1049.834473,710.336914,1045.975220,709.147095,1043.568481
5898,580.772339,1508.721436,581.440552,1507.762329,581.395081,1506.460693,581.781494,1505.273560,582.028503,1504.037598,...,583.355347,1478.002563,583.237732,1476.652588,583.116028,1475.285889,582.995117,1473.909912,582.880310,1472.534302
22731,844.438110,1483.102295,845.010803,1482.036011,844.971008,1480.514404,845.166321,1479.039795,845.310791,1477.525146,...,845.165771,1448.065796,845.093079,1446.598389,845.045044,1445.166016,845.029236,1443.787476,845.051208,1442.479248
30778,697.559387,1155.117554,697.089050,1155.274780,696.596863,1154.771851,696.224792,1154.505371,695.816040,1154.127563,...,682.771790,1146.225952,682.203003,1145.915771,681.636169,1145.608643,681.070923,1145.304688,680.507019,1145.004150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13384,1985.088013,565.052429,1987.104980,566.986084,1988.788086,568.257019,1990.203247,569.365234,1991.524414,570.417297,...,2014.698242,588.320923,2015.656128,589.130920,2016.532593,589.974670,2017.312012,590.852539,2017.988647,591.754150
26761,1887.142822,479.195312,1889.718384,481.436554,1891.558716,482.803467,1893.056030,483.962555,1894.432129,485.052551,...,1917.610352,503.395935,1918.298828,504.275909,1918.894775,505.185913,1919.419800,506.101379,1919.959839,507.062469
4230,253.128311,1014.088257,253.995163,1012.951233,253.910614,1010.948914,253.922363,1008.939880,253.943207,1006.916565,...,255.117432,973.873291,255.393677,972.953003,255.684769,972.134766,255.972412,971.373535,256.193604,970.408813


In [12]:
df2.to_csv("outputs6epb4all_batchnormEncoder.csv", index=True)